In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
from peft import PeftModel
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find cuobjdump.exe
  warnings.warn(f"Failed to find {binary}")
c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find nvdisasm.exe
  warnings.warn(f"Failed to find {binary}")
Skipping import of cpp extensions due to incompatible torch version 2.9.1+cu126 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
W1126 22:28:32.028000 42252 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
def clean_input_text(text):
    text = re.sub(r"^(ID|EN|To|From)\s*:\s*", "", text, flags=re.I)
    return text.strip()

In [ ]:


device = "cuda"
model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map=device
)
lora_path = "./QWEN3"  

model = PeftModel.from_pretrained(
    model,
    lora_path,
    device_map=device
)

model = model.eval()

def prompt_role_based(text, domain=None):
    domain_info = f"({domain})" if domain else ""

    return f"""
Translate the English sentence to Indonesian {domain_info}.
This is a direct mapping task. No reasoning. No explanation.
Do not modify meaning. Do not invert comparative direction.
cheaper → lebih murah
more expensive → lebih mahal

Return ONLY the Indonesian translation.

English: {text}
Indonesian:
""".strip()


def strip_wrappers(s: str):
    if not isinstance(s, str):
        return ""
    s = s.strip()
    s = re.sub(r"^\[\s*[\"'](.*?)[\"']\s*\]$", r"\1", s)
    s = s.strip('"').strip("'")

    return s.strip()


def generate_text(prompt, max_new_tokens=60):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.0,
            do_sample=False,
            top_p=1.0
        )

    return tokenizer.decode(out[0], skip_special_tokens=True)

def extract_translation(full_output, prompt):
    gen = full_output.replace(prompt, "").strip()
    if "Indonesian:" in gen:
        gen = gen.split("Indonesian:")[1].strip()
    sentence = gen.split(".")[0].strip()
    return sentence

def translate_role(text, domain=None):
    text = clean_input_text(text)   
    prompt = prompt_role_based(text, domain=domain)
    output = generate_text(prompt, max_new_tokens=160)
    return extract_translation(output, prompt)

english_sentence = "The auditors concluded that the financial discrepancies did not stem from intentional misreporting, but from a series of legacy processes that were never fully integrated during the previous restructuring."
result = translate_role(english_sentence, domain="audit")
print("EN:", english_sentence)
print("ID:", result)


`torch_dtype` is deprecated! Use `dtype` instead!
c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\modeling_utils.py:6105: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  _ = torch.empty(byte_count // factor, dtype=torch.float16, device=device, requires_grad=False)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\peft\config.py:162: UserWarning: Unexpected keyword arguments ['alora_invocation_tokens', 'arrow_config', 'corda_config', 'ensure_weight_tying', 'peft_version', 'qalora_group_size', 'target_parameters', 'trainable_token_indices', 'use_qalora'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(
The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


EN: The auditors concluded that the financial discrepancies did not stem from intentional misreporting, but from a series of legacy processes that were never fully integrated during the previous restructuring.
ID: Auditernya menutup bahwa perbedaan keuangan tidak berasal dari penyebaran yang sengaja, tetapi dari sejumlah proses legi yang tidak pernah sepenuhnya terintegrasi selama restructuring sebelumnya
